# Projet SOIA 6.1 - PINN
## Oscar Agnus, SOIA A3


Here, we try to simulate the 1D heat equation using a physics-informed neural network (PINN). The heat equation is given by:
$$
\frac{\partial u}{\partial t} = \nu \frac{\partial^2 u}{\partial x^2}
$$

where $u$ is the temperature,
$x$ is the space coordinate,
$t$ is the time coordinate,
and $\nu$ is the thermal diffusivity.


In [1]:
# Imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import pinnde.pde_Solvers as pde_Solvers
import pinnde.pde_Initials as pde_Initials
import pinnde.pde_Boundaries_2var as pde_Boundaries

Check in the PC configuration if the GPU is available. 

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if len(tf.config.list_physical_devices('GPU')) == 0:
    print("No GPU was detected. The training will be done on the CPU.")
else:
    print("GPU detected. The training will be done on the GPU.")

Num GPUs Available:  0
No GPU was detected. The training will be done on the CPU.


### Setting up the problem

In [3]:
# Code extracted from "PinnDE: Physics-Informed Neural Networks for Solving Differential Equations" by Jason Mathews and Alex Bihlo.
# and slightly remixed by Oscar Agnus

# Parameters
nu = 0.1 # Thermal diffusivity

N_bc = 100 # Number of boundary conditions
N_pde = 10000 # Number of PDE conditions
N_sensors = 30000 # Number of sensors
N_iv = 100 # Number of initial values

# Boundaries of the domain
t_bdry = [0,1]
x_bdry = [0,1]
t_order = 1

# Initial conditions
u0 = lambda x: tf.sin(np.pi*x)
# u0 = lambda x: np.zeros_like(x)

initials = pde_Initials.setup_initials_2var(t_bdry, x_bdry, t_order, [u0], N_iv)

# Boundary conditions
all_boundary = lambda x: 0+x*0
boundaries = pde_Boundaries.setup_boundaries_dirichlet_tx(t_bdry, x_bdry, N_bc, all_boundaries_cond=all_boundary)

eqn = "0.1*uxx - ut" # PDE equation (1D heat equation in this case)
sensor_range = [-2, 2]

### Machine learning model

In [4]:
# Training the model
epochs = 3000

PINN_heat_eq = pde_Solvers.solvePDE_DeepONet_tx(eqn, initials, boundaries, N_pde, N_sensors, sensor_range)
PINN_heat_eq.train_model(epochs)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize_1         │ (None, 1)         │          0 │ input_layer_1[0]… │
│ (Normalize)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalize_3         │ (None, 1)         │          0 │ input_layer_2[0]… │
│ (Normalize)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2)         │          0 │ normalize_1[0][0… │
│ (Concatenate)       │                   │            │ normalize_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 60)        │      6,060 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 60)        │        180 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 60)        │      3,660 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 60)        │      3,660 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 60)        │      3,660 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 60)        │      3,660 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 60)        │      3,660 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 60)        │      3,660 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 60)        │          0 │ dense_5[0][0],    │
│                     │                   │            │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 1)         │         61 │ multiply[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,261 (220.79 KB)

 Trainable params: 28,261 (220.79 KB)

 Non-trainable params: 0 (0.00 B)

Total loss 0th epoch:  0.2331.
Total loss 100th epoch:  0.0001.
Total loss 200th epoch:  0.0000.
Total loss 300th epoch:  0.0000.
Total loss 400th epoch:  0.0000.
Total loss 500th epoch:  0.0000.
Total loss 600th epoch:  0.0000.
Total loss 700th epoch:  0.0000.
Total loss 800th epoch:  0.0000.
Total loss 900th epoch:  0.0000.
Total loss 1000th epoch:  0.0000.
Total loss 1100th epoch:  0.0000.
Total loss 1200th epoch:  0.0000.
Total loss 1300th epoch:  0.0000.
Total loss 1400th epoch:  0.0000.
Total loss 1500th epoch:  0.0000.
Total loss 1600th epoch:  0.0000.
Total loss 1700th epoch:  0.0000.
Total loss 1800th epoch:  0.0000.
Total loss 1900th epoch:  0.0000.
Total loss 2000th epoch:  0.0000.
Total loss 2100th epoch:  0.0000.
Total loss 2200th epoch:  0.0000.
Total loss 2300th epoch:  0.0000.
Total loss 2400th epoch:  0.0000.
Total loss 2500th epoch:  0.0000.
Total loss 2600th epoch:  0.0000.
Total loss 2700th epoch:  0.0000.
Total loss 2800th epoch:  0.0000.
Total loss 2900th epoch:  

In [12]:
# Evaluate the model
exact_eqn = "np.e**(-((np.pi**2) * 0.1 * t))*np.sin(np.pi*x)" # Exact solution
PINN_heat_eq.plot_predicted_exact(exact_eqn)

<Figure size 1400x500 with 0 Axes>

In [6]:
# Display the results
